In [105]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated

In [106]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/LAB3-20221109T160610Z-001/LCL-FullData/Cluster14data.csv')

In [109]:
# dividing the two year data into 24 parts (each 1 month data)
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [110]:
# Filter the DataFrame to include readings from Jan 01, 2012, to Dec 31, 2012
start_date1 = pd.to_datetime('2012-01-01')
end_date1 = pd.to_datetime('2012-12-31')
filtered_data1 = data[(data['DateTime'] >= start_date1) & (data['DateTime'] <= end_date1)]

# Filter the DataFrame to include readings from Jan 01, 2013, to Dec 31, 2013
start_date2 = pd.to_datetime('2013-01-01')
end_date2 = pd.to_datetime('2013-12-31')
filtered_data2 = data[(data['DateTime'] >= start_date2) & (data['DateTime'] <= end_date2)]

In [111]:
# Select 5 unique LCLids
lclid_list1 = filtered_data1['LCLid'].unique()
selected_lclids1 = lclid_list1[:20]

# Filter data for the selected LCLids
f_data1 = filtered_data1[filtered_data1['LCLid'].isin(selected_lclids1)]

# Select 5 unique LCLids
lclid_list2 = filtered_data2['LCLid'].unique()
selected_lclids2 = lclid_list2[:20]

# Filter data for the selected LCLids
f_data2 = filtered_data2[filtered_data2['LCLid'].isin(selected_lclids2)]

In [112]:
datan1 = f_data1.copy()
datan1['KWH/hh'] = datan1['KWH/hh'].astype(np.float32)
datan2 = f_data2.copy()
datan2['KWH/hh'] = datan2['KWH/hh'].astype(np.float32)

datan1 = datan1.drop('cluster', axis=1)
datan1 = datan1.drop('stdorToU', axis=1)
datan2 = datan2.drop('cluster', axis=1)
datan2 = datan2.drop('stdorToU', axis=1)

In [113]:
datan1.reset_index(drop=True, inplace=True)
datan2.reset_index(drop=True, inplace=True)

In [114]:
datan1['DateTime'] = pd.to_datetime(datan1.DateTime).dt.tz_localize(None)
for i in range(len(datan1)):
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()

<ipython-input-114-b59380e28e58>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datan1['DateTime'][i]=datan1['DateTime'][i].timestamp()


In [115]:
datan2['DateTime'] = pd.to_datetime(datan2.DateTime).dt.tz_localize(None)
for i in range(len(datan2)):
  datan2['DateTime'][i]=datan2['DateTime'][i].timestamp()

<ipython-input-115-04c0d2c83489>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datan2['DateTime'][i]=datan2['DateTime'][i].timestamp()


In [116]:
datan1['DateTime'] = datan1['DateTime'].astype(np.float32)
datan2['DateTime'] = datan2['DateTime'].astype(np.float32)

In [117]:
# Sort the data by 'LCLid' and 'DateTime'
datan1.sort_values(['LCLid', 'DateTime'], inplace=True)
datan2.sort_values(['LCLid', 'DateTime'], inplace=True)

In [118]:
# Define the client window dataset function for a specific LCLid
def create_client_dataset_for_LCLid(client_data, window_size, step_size):
    client_windows = []
    client_targets = []
    num_readings = len(client_data)
    
    # Iterate over the readings using the sliding window
    for i in range(0, num_readings - window_size, step_size):
        window_start = i 
        window_end = i + window_size - 1
        prediction_index = window_end + step_size
        
        # Extract the window and the prediction target
        window = client_data.iloc[window_start:window_end + 1]['KWH/hh'].values
        target = client_data.iloc[prediction_index]['KWH/hh']
        
        client_windows.append(window)
        client_targets.append(target)
    
    # Create an ordered dictionary with 'x' and 'y' keys
    ordered_dict = collections.OrderedDict()
    ordered_dict['x'] = tf.stack(client_windows)
    ordered_dict['y'] = tf.expand_dims(client_targets, axis=-1)
    
    
    return ordered_dict

In [119]:
window_size = 336
step_size = 1

# Filter the dataframe for the specific LCLid
example_LCLid = datan1['LCLid'].unique()[3]
clientyy_data = datan1[datan1['LCLid'] == example_LCLid]

# Create the client dataset for the specific LCLid
example_client_dataset = create_client_dataset_for_LCLid(clientyy_data, window_size, step_size)

print("Client dataset for LCLid", example_LCLid)
print(example_client_dataset)

Client dataset for LCLid MAC000200
OrderedDict([('x', <tf.Tensor: shape=(17193, 336), dtype=float32, numpy=
array([[0.013, 0.012, 0.019, ..., 0.077, 0.08 , 0.118],
       [0.012, 0.019, 0.063, ..., 0.08 , 0.118, 0.123],
       [0.019, 0.063, 0.013, ..., 0.118, 0.123, 0.015],
       ...,
       [0.115, 0.078, 0.106, ..., 0.59 , 0.073, 0.117],
       [0.078, 0.106, 0.016, ..., 0.073, 0.117, 0.112],
       [0.106, 0.016, 0.029, ..., 0.117, 0.112, 0.089]], dtype=float32)>), ('y', <tf.Tensor: shape=(17193, 1), dtype=float32, numpy=
array([[0.123],
       [0.015],
       [0.013],
       ...,
       [0.112],
       [0.089],
       [0.097]], dtype=float32)>)])


In [120]:
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess_client_dataset(dataset):
    def batch_format_fn(element):
        return collections.OrderedDict(
            x=tf.reshape(element['x'], [-1, 336]),
            y=tf.reshape(element['y'], [-1, 1]))
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
        BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
 

preprocessed_example_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(example_client_dataset))


sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_client_dataset)))

sample_batch

OrderedDict([('x',
              array([[0.013, 0.051, 0.029, ..., 0.057, 0.09 , 0.084],
                     [0.013, 0.04 , 0.041, ..., 0.013, 0.082, 0.039],
                     [0.012, 0.013, 0.046, ..., 0.148, 0.031, 0.017],
                     ...,
                     [0.062, 0.018, 0.022, ..., 0.012, 0.013, 0.012],
                     [0.042, 0.038, 0.012, ..., 0.023, 0.024, 0.024],
                     [0.081, 0.078, 0.079, ..., 0.024, 0.025, 0.025]], dtype=float32)),
             ('y',
              array([[0.148],
                     [0.254],
                     [0.063],
                     [0.226],
                     [0.027],
                     [0.042],
                     [0.275],
                     [0.599],
                     [0.017],
                     [0.079],
                     [0.051],
                     [0.321],
                     [0.065],
                     [0.062],
                     [0.013],
                     [0.07 ],
                  

In [121]:
NUM_CLIENTS = 10  # Replace with desired number of clients
sample_clients = datan1['LCLid'].unique()[0:NUM_CLIENTS]
sample_clients_list = sample_clients.tolist()
sample_clients_list

['MAC000025',
 'MAC000104',
 'MAC000160',
 'MAC000200',
 'MAC000229',
 'MAC000283',
 'MAC000297',
 'MAC000318',
 'MAC000319',
 'MAC000391']

In [122]:
# Iterate over unique LCLids in the dataframe
client_datasets_12 = {}
for LCLid in sample_clients_list:
    # Filter the dataframe for the current LCLid
    client_data = datan1[datan1['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)
    
    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))
    
    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))
    
    # Store the preprocessed dataset in the dictionary with LCLid as the key
    client_datasets_12[LCLid] = preprocessed_client_dataset

    print("Client dataset for LCLid", LCLid)
    print(sam_batch)

Client dataset for LCLid MAC000025
OrderedDict([('x', array([[0.000e+00, 2.100e-02, 2.000e-03, ..., 1.300e-02, 0.000e+00,
        2.600e-02],
       [3.200e-02, 6.900e-02, 1.709e+00, ..., 1.200e-02, 2.300e-02,
        2.500e-02],
       [4.900e-02, 6.700e-02, 6.200e-02, ..., 1.838e+00, 1.910e-01,
        6.200e-02],
       ...,
       [2.400e-02, 0.000e+00, 1.500e-02, ..., 0.000e+00, 2.400e-02,
        1.000e-03],
       [4.300e-02, 6.280e-01, 2.800e-02, ..., 1.500e-02, 1.260e-01,
        1.190e-01],
       [4.590e-01, 2.270e-01, 0.000e+00, ..., 5.700e-02, 5.980e-01,
        7.140e-01]], dtype=float32)), ('y', array([[0.   ],
       [0.067],
       [0.075],
       [0.023],
       [0.058],
       [0.   ],
       [0.021],
       [0.   ],
       [0.04 ],
       [0.098],
       [0.023],
       [0.023],
       [0.052],
       [0.598],
       [0.026],
       [0.011],
       [0.024],
       [0.002],
       [0.64 ],
       [0.063]], dtype=float32))])
Client dataset for LCLid MAC000104
OrderedD

In [123]:
# Iterate over unique LCLids in the dataframe
client_datasets_13 = {}
for LCLid in sample_clients_list:
    # Filter the dataframe for the current LCLid
    client_data = datan2[datan2['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)
    
    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))
    
    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))
    
    # Store the preprocessed dataset in the dictionary with LCLid as the key
    client_datasets_13[LCLid] = preprocessed_client_dataset



In [124]:
def make_federated_data(client_datasets, sample_clients_list):
    return [
        client_datasets[x] for x in sample_clients_list
    ]

In [125]:
federated_train_data_12 = make_federated_data(client_datasets_12, sample_clients)
federated_train_data_13 = make_federated_data(client_datasets_13, sample_clients)

In [126]:
print(f'Number of client datasets: {len(federated_train_data_12)}')
print(f'First dataset: {federated_train_data_12[0]}')
print(f'Second dataset: {federated_train_data_12[1]}')

Number of client datasets: 10
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>
Second dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>


In [127]:
preprocessed_example_client_dataset.element_spec

OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)),
             ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])

In [128]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(336,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

In [129]:
def model_fn():
    keras_model = create_model()
    loss = tf.keras.losses.MeanSquaredError() 
    tff_model = tff.learning.models.from_keras_model(
        keras_model,
        input_spec=preprocessed_example_client_dataset.element_spec,
        loss=loss,
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    return tff_model

In [130]:
model = model_fn()
print(model)

In [131]:
# training starts
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

In [132]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[336,64],
      float32[64],
      float32[64,32],
      float32[32],
      float32[32,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[336,64],
    float32[64],
    float32[64,32],
    float32[32],
    float32[32,1],
    float32[1]
  >
>@SERVER)


In [133]:
train_state = training_process.initialize()

In [134]:
federated_train_data_12

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, n

In [135]:
federated_train_data_12

[<_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))])>,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 336), dtype=tf.float32, n

In [136]:
result = training_process.next(train_state, federated_train_data_12)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.08946611), ('loss', 0.028399965), ('num_examples', 781350), ('num_batches', 39073)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [137]:
NUM_ROUNDS = 6
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data_12)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.08241468), ('loss', 0.025845122), ('num_examples', 781350), ('num_batches', 39073)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.07916815), ('loss', 0.024746662), ('num_examples', 781350), ('num_batches', 39073)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.07726406), ('loss', 0.024137653), ('num_examples', 781350), ('num_batches', 39073)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalize

In [138]:
# Filter the DataFrame to include readings from Jan 01, 2014, to Jan 31, 2014
teststart_date = pd.to_datetime('2014-01-01')
testend_date = pd.to_datetime('2014-01-31')
test_data = data[(data['DateTime'] >= start_date1) & (data['DateTime'] <= end_date1)]

In [139]:
# Select 5 unique LCLids
lclid_lt = test_data['LCLid'].unique()
sel_lclids = lclid_lt[2:3]

print(sel_lclids)
# # Filter data for the selected LCLids
ft_data = test_data[test_data['LCLid'].isin(sel_lclids)]

['MAC000104']


In [140]:
ft_data['KWH/hh'] = ft_data['KWH/hh'].astype(np.float32)

<ipython-input-140-398836f43ad2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft_data['KWH/hh'] = ft_data['KWH/hh'].astype(np.float32)


In [141]:
ft_data = ft_data.drop('cluster', axis=1)
ft_data = ft_data.drop('stdorToU', axis=1)
ft_data.reset_index(drop=True, inplace=True)

In [142]:
ft_data['DateTime'] = pd.to_datetime(ft_data.DateTime).dt.tz_localize(None)
for i in range(len(ft_data)):
  ft_data['DateTime'][i]=ft_data['DateTime'][i].timestamp()

<ipython-input-142-fe80fcc7ed1b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ft_data['DateTime'][i]=ft_data['DateTime'][i].timestamp()


In [143]:
ft_data['DateTime'] = ft_data['DateTime'].astype(np.float32)

In [144]:
# Sort the data by 'LCLid' and 'DateTime'
ft_data.sort_values(['LCLid', 'DateTime'], inplace=True)

In [145]:
ft_data

,LCLid,DateTime,KWH/hh
0,MAC000104,1.325376e+09,0.051
1,MAC000104,1.325378e+09,0.052
2,MAC000104,1.325380e+09,0.136
3,MAC000104,1.325381e+09,0.151
4,MAC000104,1.325383e+09,0.112
...,...,...,...
17521,MAC000104,1.356905e+09,0.282
17522,MAC000104,1.356907e+09,0.311
17523,MAC000104,1.356908e+09,0.297
17524,MAC000104,1.356910e+09,0.242


In [146]:
test_client = ft_data['LCLid'].unique()
test_client 

array(['MAC000104'], dtype=object)

In [147]:
test_dataset = {}

for LCLid in test_client:
    # Filter the dataframe for the current LCLid
    client_data = ft_data[ft_data['LCLid'] == LCLid]

    clientxx_dataset = create_client_dataset_for_LCLid(client_data, window_size, step_size)
    
    # Create the client dataset for the current LCLid
    preprocessed_client_dataset = preprocess_client_dataset(tf.data.Dataset.from_tensor_slices(clientxx_dataset))
    
    # Extract a sample batch from the preprocessed dataset
    sam_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_client_dataset)))
    
    # Store the preprocessed dataset in the dictionary with LCLid as the key
    test_dataset[LCLid] = preprocessed_client_dataset

    print("Client dataset for LCLid", LCLid)
    print(sam_batch)

Client dataset for LCLid MAC000104
OrderedDict([('x', array([[0.054, 0.105, 0.119, ..., 0.083, 0.082, 0.043],
       [0.162, 0.423, 0.479, ..., 0.372, 0.31 , 0.366],
       [1.283, 0.537, 0.398, ..., 0.43 , 0.42 , 0.327],
       ...,
       [0.123, 0.103, 0.099, ..., 0.098, 0.097, 0.097],
       [0.045, 0.113, 0.123, ..., 0.096, 0.098, 0.098],
       [0.272, 0.132, 0.108, ..., 0.177, 0.218, 0.194]], dtype=float32)), ('y', array([[0.033],
       [0.558],
       [0.473],
       [0.083],
       [0.097],
       [0.134],
       [0.337],
       [0.174],
       [0.366],
       [0.033],
       [0.066],
       [0.366],
       [0.085],
       [0.096],
       [0.11 ],
       [0.31 ],
       [0.275],
       [0.074],
       [0.097],
       [0.371]], dtype=float32))])


In [148]:
# Make federated test data
federated_test_data = make_federated_data(test_dataset, test_client)

In [149]:
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

In [150]:
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[336,64],
        float32[64],
        float32[64,32],
        float32[32],
        float32[32,1],
        float32[1]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        mean_absolute_error=<
          float32,
          float32
        >,
        loss=<
          float32,
          float32
        >,
        num_examples=<
          int64
        >,
        num_batches=<
          int64
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    x=float32[?,336],
    y=float32[?,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[336,64],
        float32[64],
        float32[64,32],
        float32[32],
        float32[32,1],
        float32[1]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <

In [151]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [152]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [153]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('mean_absolute_error', 0.08888001), ('loss', 0.02648883), ('num_examples', 85950), ('num_batches', 4298)])), ('total_rounds_metrics', OrderedDict([('mean_absolute_error', 0.08888001), ('loss', 0.02648883), ('num_examples', 85950), ('num_batches', 4298)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

In [154]:
NUM_ROUNDS = 6
for round_num in range(1, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data_13)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.076887876), ('loss', 0.023536151), ('num_examples', 857185), ('num_batches', 42862)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.07556862), ('loss', 0.023088805), ('num_examples', 857185), ('num_batches', 42862)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('mean_absolute_error', 0.07469604), ('loss', 0.022808697), ('num_examples', 857185), ('num_batches', 42862)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finaliz

In [155]:
evaluation_process2 = tff.learning.algorithms.build_fed_eval(model_fn)

In [156]:
evaluation_state2 = evaluation_process2.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state2 = evaluation_process2.set_model_weights(evaluation_state2, model_weights)

In [157]:
evaluation_output2 = evaluation_process2.next(evaluation_state2, federated_test_data)

In [158]:
str(evaluation_output2.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('mean_absolute_error', 0.0828848), ('loss', 0.025040235), ('num_examples', 85950), ('num_batches', 4298)])), ('total_rounds_metrics', OrderedDict([('mean_absolute_error', 0.0828848), ('loss', 0.025040235), ('num_examples', 85950), ('num_batches', 4298)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

In [159]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('mean_absolute_error', 0.08888001), ('loss', 0.02648883), ('num_examples', 85950), ('num_batches', 4298)])), ('total_rounds_metrics', OrderedDict([('mean_absolute_error', 0.08888001), ('loss', 0.02648883), ('num_examples', 85950), ('num_batches', 4298)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"